In [7]:
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Suprime avisos para uma saída mais limpa
warnings.filterwarnings('ignore')

# Configuração
DATASET_FILENAME = 'agaricus-lepiota.data'
PROCESSED_FILENAME = 'mushroom_processed_final.csv'

COLUMNS = [
    'class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
    'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape',
    'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring',
    'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type',
    'veil-color', 'ring-number', 'ring-type', 'spore-print-color',
    'population', 'habitat'
]

def load_and_preprocess_data():
    """
    Carrega o dataset, trata valores ausentes descartando linhas (abordagem conservadora),
    e aplica One-Hot Encoding.

    Retorna:
        X (pd.DataFrame): Matriz de características.
        y (pd.Series): Vetor alvo.
    """
    # 1. Carregamento
    if not os.path.exists(DATASET_FILENAME):
        print(f"O arquivo {DATASET_FILENAME} não foi encontrado. Certifique-se de que está no diretório correto.")
        return None, None

    df = pd.read_csv(DATASET_FILENAME, header=None, names=COLUMNS, na_values='?')
    initial_shape = df.shape
    print(f"Formato original do dataset: {initial_shape}")

    # 2. Tratamento de Valores Ausentes (Abordagem Conservadora)
    # Descartando linhas com valores ausentes para manter a integridade dos dados
    df_clean = df.dropna().reset_index(drop=True)
    dropped_rows = initial_shape[0] - df_clean.shape[0]
    print(f"Linhas descartadas devido a valores ausentes: {dropped_rows} ({(dropped_rows/initial_shape[0]):.2%})")

    # 3. Codificação
    X = df_clean.drop('class', axis=1)
    y = df_clean['class']

    # Codifica o Alvo (0=comestível, 1=venenoso)
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Codifica as Características (One-Hot Encoding)
    X_encoded = pd.get_dummies(X)

    print(f"Formato do dataset processado: {X_encoded.shape}")

    # Salva para registro
    df_final = pd.concat([pd.DataFrame(y_encoded, columns=['target']), X_encoded], axis=1)
    df_final.to_csv(PROCESSED_FILENAME, index=False)

    return X_encoded, y_encoded

def get_models_configuration():
    """Define os modelos e grades de hiperparâmetros."""
    return [
        {
            'name': 'Árvore de Decisão',
            'model': DecisionTreeClassifier(random_state=42),
            'params': {
                'criterion': ['gini', 'entropy'],
                'max_depth': [None, 10, 5],
                'min_samples_split': [2, 5]
            }
        },
        {
            'name': 'KNN',
            'model': KNeighborsClassifier(),
            'params': {
                'n_neighbors': [3, 5, 7],
                'weights': ['uniform', 'distance'],
                'metric': ['euclidean', 'manhattan']
            }
        },
        {
            'name': 'Naive Bayes',
            'model': GaussianNB(),
            'params': {
                'var_smoothing': [1e-9, 1e-8, 1e-7]
            }
        },
        {
            'name': 'Regressão Logística',
            'model': LogisticRegression(max_iter=1000, random_state=42),
            'params': {
                'C': [0.1, 1, 10],
                'solver': ['liblinear', 'lbfgs']
            }
        },
        {
            'name': 'Rede Neural MLP',
            'model': MLPClassifier(max_iter=500, random_state=42),
            'params': {
                'hidden_layer_sizes': [(50,), (100,), (50, 50)],
                'activation': ['relu', 'tanh'],
                'alpha': [0.0001, 0.001]
            }
        }
    ]

def _run_single_fold_evaluation(model_config, X_train, y_train, X_test, y_test):
    """
    Função auxiliar para executar GridSearchCV e avaliar um modelo para uma única dobra.
    Retorna best_params, acurácia, precisão, recall, f1.
    """
    grid_search = GridSearchCV(
        model_config['model'],
        model_config['params'],
        scoring='accuracy',
        cv=3,
        n_jobs=-1,
        refit=True
    )
    grid_search.fit(X_train, y_train)

    best_params = str(grid_search.best_params_)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    return best_params, acc, prec, rec, f1

def run_nested_cross_validation(X, y, n_splits=10):
    """
    Realiza Validação Cruzada Aninhada para prevenir vazamento de dados durante o ajuste de hiperparâmetros.
    """
    cv_outer = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    models_config = get_models_configuration()
    results_summary = []

    print("\n" + "="*80)
    print(f"{'INICIANDO VALIDAÇÃO CRUZADA ANINHADA (10-FOLD)':^80}")
    print("="*80)

    for config in models_config:
        model_name = config['name']
        print(f"\nProcessando Modelo: {model_name}...")

        fold_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
        best_params_tracker = []

        # Loop Externo: Avaliação
        for fold_idx, (train_ix, test_ix) in enumerate(cv_outer.split(X, y)):
            X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
            y_train, y_test = y[train_ix], y[test_ix]

            # Executa o GridSearchCV e avalia o modelo para esta dobra
            best_params, acc, prec, rec, f1 = _run_single_fold_evaluation(
                config, X_train, y_train, X_test, y_test
            )

            # Armazena os melhores parâmetros para esta dobra
            best_params_tracker.append(best_params)

            # Adiciona as métricas para esta dobra
            fold_metrics['accuracy'].append(acc)
            fold_metrics['precision'].append(prec)
            fold_metrics['recall'].append(rec)
            fold_metrics['f1'].append(f1)

        # Agregando resultados
        mean_acc = np.mean(fold_metrics['accuracy'])
        std_acc = np.std(fold_metrics['accuracy'])
        mean_f1 = np.mean(fold_metrics['f1'])
        std_f1 = np.std(fold_metrics['f1'])

        # Encontra os parâmetros mais comuns em todas as dobras
        from collections import Counter
        most_common_params = Counter(best_params_tracker).most_common(1)[0][0]

        print(f"   Acurácia: {mean_acc:.4f} (+/- {std_acc:.4f})")
        print(f"   Pontuação F1: {mean_f1:.4f}")

        results_summary.append({
            'Algorithm': model_name,
            'Accuracy': f"{mean_acc:.4f} ± {std_acc:.4f}",
            'Precision': f"{np.mean(fold_metrics['precision']):.4f} ± {np.std(fold_metrics['precision']):.4f}",
            'Recall': f"{np.mean(fold_metrics['recall']):.4f} ± {np.std(fold_metrics['recall']):.4f}",
            'F1-Score': f"{mean_f1:.4f} ± {std_f1:.4f}",
            'Best Params (Mode)': most_common_params
        })

    return pd.DataFrame(results_summary)

if __name__ == "__main__":
    # 1. Pré-processamento
    X, y = load_and_preprocess_data()

    if X is not None:
        # 2. Execução
        df_results = run_nested_cross_validation(X, y)

        # 3. Relatório Final
        print("\n" + "="*80)
        print(f"{'RELATÓRIO FINAL DE DESEMPENHO':^80}")
        print("="*80)
        print(df_results.drop('Best Params (Mode)', axis=1).to_markdown(index=False))

        # Salva resultados detalhados
        df_results.to_csv('final_model_evaluation.csv', index=False)
        print("\nResultados detalhados salvos em 'final_model_evaluation.csv'.")

Formato original do dataset: (8124, 23)
Linhas descartadas devido a valores ausentes: 2480 (30.53%)
Formato do dataset processado: (5644, 98)

                 INICIANDO VALIDAÇÃO CRUZADA ANINHADA (10-FOLD)                 

Processando Modelo: Árvore de Decisão...
   Acurácia: 1.0000 (+/- 0.0000)
   Pontuação F1: 1.0000

Processando Modelo: KNN...
   Acurácia: 1.0000 (+/- 0.0000)
   Pontuação F1: 1.0000

Processando Modelo: Naive Bayes...
   Acurácia: 0.9981 (+/- 0.0017)
   Pontuação F1: 0.9975

Processando Modelo: Regressão Logística...
   Acurácia: 1.0000 (+/- 0.0000)
   Pontuação F1: 1.0000

Processando Modelo: Rede Neural MLP...
   Acurácia: 1.0000 (+/- 0.0000)
   Pontuação F1: 1.0000

                         RELATÓRIO FINAL DE DESEMPENHO                          
| Algorithm           | Accuracy        | Precision       | Recall          | F1-Score        |
|:--------------------|:----------------|:----------------|:----------------|:----------------|
| Árvore de Decisão   | 1.0

In [11]:
import pandas as pd

# 1. Carregar o arquivo de resultados já gerado
try:
    df = pd.read_csv('final_model_evaluation.csv')

    print("="*80)
    print("TABELA 1: MELHORES HIPERPARÂMETROS (Copie as linhas abaixo para o Word)")
    print("="*80)
    print(f"| {'Algorithm':<20} | {'Melhores Hiperparâmetros (Moda)':<50} |")
    print(f"|{'-'*22}|{'-'*52}|")

    # 2. Iterar e formatar para ficar bonito no artigo
    for index, row in df.iterrows():
        # Pega a coluna 'Best Params (Mode)' ou 'Melhores Params (Moda)' dependendo da versão que rodou
        params = row.get('Best Params (Mode)', row.get('Melhores Params (Moda)', 'Coluna não encontrada'))

        # Limpeza visual para o artigo (tira chaves e aspas)
        params_limpo = str(params).replace("{", "").replace("}", "").replace("'", "")

        print(f"| {row['Algorithm']:<20} | {params_limpo:<50} |")

except FileNotFoundError:
    print("ERRO: O arquivo 'final_model_evaluation.csv' não foi encontrado.")
    print("Verifique se você rodou o código de treinamento anterior completamente.")

TABELA 1: MELHORES HIPERPARÂMETROS (Copie as linhas abaixo para o Word)
| Algorithm            | Melhores Hiperparâmetros (Moda)                    |
|----------------------|----------------------------------------------------|
| Árvore de Decisão    | criterion: gini, max_depth: None, min_samples_split: 2 |
| KNN                  | metric: manhattan, n_neighbors: 7, weights: distance |
| Naive Bayes          | var_smoothing: 1e-09                               |
| Regressão Logística  | C: 10, solver: lbfgs                               |
| Rede Neural MLP      | activation: tanh, alpha: 0.0001, hidden_layer_sizes: (100,) |
